In [31]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [32]:
movies = pd.read_csv("movies.csv")
movies.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [33]:
selected_features = ['genres','director','cast','keywords','tagline','original_title','original_language']

for features in selected_features:
  movies[features]=movies[features].fillna('')


In [34]:
required_features=movies['genres']+' '+movies['director']+' '+movies['cast']+' '+movies['keywords']+' '+movies['tagline']+' '+movies['original_title']
print(required_features)


0       Action Adventure Fantasy Science Fiction James...
1       Adventure Fantasy Action Gore Verbinski Johnny...
2       Action Adventure Crime Sam Mendes Daniel Craig...
3       Action Crime Drama Thriller Christopher Nolan ...
4       Action Adventure Science Fiction Andrew Stanto...
                              ...                        
4798    Action Crime Thriller Robert Rodriguez Carlos ...
4799    Comedy Romance Edward Burns Edward Burns Kerry...
4800    Comedy Drama Romance TV Movie Scott Smith Eric...
4801     Daniel Hsia Daniel Henney Eliza Coupe Bill Pa...
4802    Documentary Brian Herzlinger Drew Barrymore Br...
Length: 4803, dtype: object


In [35]:
vector=TfidfVectorizer()
vector_features = vector.fit_transform(required_features)


In [36]:
similarity = cosine_similarity(vector_features)#to compare each feature row with each other

In [37]:
similarity

array([[1.        , 0.07355546, 0.03531556, ..., 0.        , 0.        ,
        0.        ],
       [0.07355546, 1.        , 0.02790734, ..., 0.04430756, 0.        ,
        0.        ],
       [0.03531556, 0.02790734, 1.        , ..., 0.        , 0.04635788,
        0.        ],
       ...,
       [0.        , 0.04430756, 0.        , ..., 1.        , 0.        ,
        0.05548941],
       [0.        , 0.        , 0.04635788, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05548941, 0.        ,
        1.        ]])

In [38]:
all_titles = movies['title'].tolist()

In [39]:
user_title = input("Enter the movie on which you would like to get rcomendations = ")

Enter the movie on which you would like to get rcomendations = forrest gump


In [40]:
import difflib
close_match = difflib.get_close_matches(user_title,all_titles)
print(close_match)

best_match = close_match[0]#storing 1st match found as best match

movie_index = movies[movies.title==best_match]['index'].values[0]
print(movie_index)

similarity_score = list(enumerate(similarity[movie_index]))
# similarity_score finding similarity of the movie in movie_index with every entry

#sort in decending order movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
# print(sorted_similar_movies)

# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 0

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies[movies.index==index]['title'].values[0]
    if (i<30):
        print(i+1, '.',title_from_index)
        i+=1


['Forrest Gump']
809
Movies suggested for you : 

1 . Forrest Gump
2 . The Green Mile
3 . A Home at the End of the World
4 . Apocalypse Now
5 . The Big Bounce
6 . The Deer Hunter
7 . Rain Man
8 . You've Got Mail
9 . Running Forever
10 . Cast Away
11 . Snake Eyes
12 . Say It Isn't So
13 . Robin and Marian
14 . Contact
15 . The Purge: Election Year
16 . Apollo 13
17 . The Musketeer
18 . Dancin' It's On
19 . Larry Crowne
20 . A Christmas Carol
21 . What's Eating Gilbert Grape
22 . Days of Heaven
23 . Impostor
24 . Letters from Iwo Jima
25 . Silver Linings Playbook
26 . The Conversation
27 . Saving Private Ryan
28 . Big
29 . Last Orders
30 . Dangerous Liaisons
